# Vertical FL Example

In [48]:
!pip install metaflow==2.4.1
!pip install ray

In [49]:
from openfl.experimental.interface import FLSpec, Aggregator, Collaborator
from openfl.experimental.runtime import LocalRuntime
from openfl.experimental.placement import aggregator, collaborator

In [50]:
class VerticalFlow(FLSpec):

    @aggregator
    def start(self):
        self.collaborators = self.runtime.collaborators
        self.round = 0
        self.next_collaborator = ['Portland']
        self.next(self.custom_task_portland, foreach='next_collaborator')

    @collaborator
    def custom_task_portland(self):
        print(f'Collaborator {self.input}: performing custom task')
        self.result = 0
        self.next(self.gather_portland_results)

    @aggregator
    def gather_portland_results(self,inputs):
        self.results = []
        self.results.append(inputs[0].result)
        self.next_collaborator = ['Seattle']
        self.next(self.custom_task_seattle, foreach='next_collaborator', exclude=['results'])

    @collaborator
    def custom_task_seattle(self):
        print(f'Collaborator {self.input}: performing custom task')
        self.result = 1
        self.next(self.gather_seattle_results)

    @aggregator
    def gather_seattle_results(self,inputs):
        self.results.append(inputs[0].result)
        self.next_collaborator = ['Chandler']
        self.next(self.custom_task_chandler, foreach='next_collaborator', exclude=['results'])

    @collaborator
    def custom_task_chandler(self):
        print(f'Collaborator {self.input}: performing custom task')
        self.result = 2
        self.next(self.gather_chandler_results)

    @aggregator
    def gather_chandler_results(self,inputs):
        self.results.append(inputs[0].result)
        self.next_collaborator = ['Bangalore']
        self.next(self.custom_task_bangalore, foreach='next_collaborator', exclude=['results'])

    @collaborator
    def custom_task_bangalore(self):
        print(f'Collaborator {self.input}: performing custom task')
        self.result = 3
        self.next(self.gather_bangalore_results)

    @aggregator
    def gather_bangalore_results(self,inputs):
        self.results.append(inputs[0].result)
        self.next(self.combine)

    @aggregator
    def combine(self):
        print(f'The results from each of the collaborators are: {self.results}')
        print(f'Their average = {sum(self.results) / len(self.results)}')
        self.round += 1
        if self.round < 10:
            print()
            print(f'Starting round {self.round}...')
            self.next_collaborator = ['Portland']
            self.next(self.custom_task_portland,foreach='next_collaborator')
        else:
            self.next(self.end)

    @aggregator
    def end(self):
        print(f'This is the end of the flow')

Aggregator step "start" registered
Collaborator step "custom_task_portland" registered
Aggregator step "gather_portland_results" registered
Collaborator step "custom_task_seattle" registered
Aggregator step "gather_seattle_results" registered
Collaborator step "custom_task_chandler" registered
Aggregator step "gather_chandler_results" registered
Collaborator step "custom_task_bangalore" registered
Aggregator step "gather_bangalore_results" registered
Aggregator step "combine" registered
Aggregator step "end" registered


In [51]:
# Setup participants
aggregator = Aggregator()
aggregator.private_attributes = {}

# Setup collaborators with private attributes
collaborator_names = ['Portland', 'Seattle', 'Chandler', 'Bangalore']
collaborators = [Collaborator(name=name) for name in collaborator_names]

local_runtime = LocalRuntime(
    aggregator=aggregator, collaborators=collaborators)
print(f'Local runtime collaborators = {local_runtime._collaborators}')

vflow = VerticalFlow(checkpoint=True)
vflow.runtime = local_runtime
vflow.run()
print(f'Reached end of the flow with collaborator results = {vflow.results}')

Local runtime collaborators = {'Portland': <openfl.experimental.interface.participants.Collaborator object at 0x7fb1fe9db9a0>, 'Seattle': <openfl.experimental.interface.participants.Collaborator object at 0x7fb1fe9db7f0>, 'Chandler': <openfl.experimental.interface.participants.Collaborator object at 0x7fb1fe9db5e0>, 'Bangalore': <openfl.experimental.interface.participants.Collaborator object at 0x7fb1fe9db700>}
Created flow VerticalFlow

Calling start
Saving data artifacts for start
Saved data artifacts for start
Sending state from aggregator to collaborators
Next function = gather_portland_results

Calling gather_portland_results
Saving data artifacts for gather_portland_results
Saved data artifacts for gather_portland_results
Sending state from aggregator to collaborators
Next function = gather_seattle_results

Calling gather_seattle_results
(wrapper pid=21844) 
(wrapper pid=21844) Running custom_task_portland in a new process
(wrapper pid=21844) 
(wrapper pid=21844) Calling custom_t

Saving data artifacts for combine
Saved data artifacts for combine
Sending state from aggregator to collaborators
(wrapper pid=21844) 
(wrapper pid=21844) Running custom_task_portland in a new process
(wrapper pid=21844) 
(wrapper pid=21844) Calling custom_task_portland
(wrapper pid=21844) Collaborator Portland: performing custom task
(wrapper pid=21844) Saving data artifacts for custom_task_portland
Next function = gather_portland_results

Calling gather_portland_results
Saving data artifacts for gather_portland_results
Saved data artifacts for gather_portland_results
Sending state from aggregator to collaborators
Next function = gather_seattle_results

Calling gather_seattle_results
(wrapper pid=21844) Saved data artifacts for custom_task_portland
(wrapper pid=21844) Should transfer from custom_task_portland to gather_portland_results
(wrapper pid=21844) 
(wrapper pid=21844) Running custom_task_seattle in a new process
(wrapper pid=21844) 
(wrapper pid=21844) Calling custom_task_seat

(wrapper pid=21844) 
(wrapper pid=21844) Running custom_task_chandler in a new process
(wrapper pid=21844) 
(wrapper pid=21844) Calling custom_task_chandler
(wrapper pid=21844) Collaborator Chandler: performing custom task
(wrapper pid=21844) Saving data artifacts for custom_task_chandler
(wrapper pid=21844) Saved data artifacts for custom_task_chandler
(wrapper pid=21844) Should transfer from custom_task_chandler to gather_chandler_results
Saving data artifacts for gather_chandler_results
Saved data artifacts for gather_chandler_results
Sending state from aggregator to collaborators
Next function = gather_bangalore_results

Calling gather_bangalore_results
(wrapper pid=21844) 
(wrapper pid=21844) Running custom_task_bangalore in a new process
(wrapper pid=21844) 
(wrapper pid=21844) Calling custom_task_bangalore
(wrapper pid=21844) Collaborator Bangalore: performing custom task
(wrapper pid=21844) Saving data artifacts for custom_task_bangalore
(wrapper pid=21844) Saved data artifacts

Saving data artifacts for gather_seattle_results
Saved data artifacts for gather_seattle_results
Sending state from aggregator to collaborators
Next function = gather_chandler_results

Calling gather_chandler_results
(wrapper pid=21844) 
(wrapper pid=21844) Running custom_task_chandler in a new process
(wrapper pid=21844) 
(wrapper pid=21844) Calling custom_task_chandler
(wrapper pid=21844) Collaborator Chandler: performing custom task
(wrapper pid=21844) Saving data artifacts for custom_task_chandler
(wrapper pid=21844) Saved data artifacts for custom_task_chandler
(wrapper pid=21844) Should transfer from custom_task_chandler to gather_chandler_results
Saving data artifacts for gather_chandler_results
Saved data artifacts for gather_chandler_results
Sending state from aggregator to collaborators
Next function = gather_bangalore_results

Calling gather_bangalore_results
(wrapper pid=21844) 
(wrapper pid=21844) Running custom_task_bangalore in a new process
(wrapper pid=21844) 
(wrapper

Now that the flow has completed, you can use metaflow tooling to look at the data associated with the experiment

In [52]:
run_id = vflow._run_id

In [53]:
import metaflow

In [54]:
from metaflow import Metaflow, Flow, Task, Step

In [55]:
m = Metaflow()
list(m)

[Flow('VerticalFlow')]

In [56]:
f = Flow('VerticalFlow').latest_run

In [57]:
f

Run('VerticalFlow/1666307288836294')

In [58]:
list(f)

[Step('VerticalFlow/1666307288836294/combine'),
 Step('VerticalFlow/1666307288836294/gather_bangalore_results'),
 Step('VerticalFlow/1666307288836294/custom_task_bangalore'),
 Step('VerticalFlow/1666307288836294/gather_chandler_results'),
 Step('VerticalFlow/1666307288836294/custom_task_chandler'),
 Step('VerticalFlow/1666307288836294/gather_seattle_results'),
 Step('VerticalFlow/1666307288836294/custom_task_seattle'),
 Step('VerticalFlow/1666307288836294/gather_portland_results'),
 Step('VerticalFlow/1666307288836294/custom_task_portland'),
 Step('VerticalFlow/1666307288836294/start')]

In [59]:
s = Step(f'VerticalFlow/{run_id}/combine')

In [60]:
s

Step('VerticalFlow/1666307288836294/combine')

In [61]:
list(s)

[Task('VerticalFlow/1666307288836294/combine/91'),
 Task('VerticalFlow/1666307288836294/combine/82'),
 Task('VerticalFlow/1666307288836294/combine/73'),
 Task('VerticalFlow/1666307288836294/combine/64'),
 Task('VerticalFlow/1666307288836294/combine/55'),
 Task('VerticalFlow/1666307288836294/combine/46'),
 Task('VerticalFlow/1666307288836294/combine/37'),
 Task('VerticalFlow/1666307288836294/combine/28'),
 Task('VerticalFlow/1666307288836294/combine/19'),
 Task('VerticalFlow/1666307288836294/combine/10')]

In [62]:
t = Task(f'VerticalFlow/{run_id}/combine/91')

In [39]:
t

Task('VerticalFlow/1666307015070672/combine/91')

In [40]:
t.data

<MetaflowData: collaborators, next_collaborator, results, round>

In [41]:
t.data.round

10